In [7]:
# Import required libraries
from langchain_community.embeddings import OllamaEmbeddings
import requests
import json

# Check if Ollama is running and what models are available
def check_ollama_status():
    """Check if Ollama is running and list available models."""
    try:
        # Check if Ollama is running
        response = requests.get("http://localhost:11434/api/tags")
        if response.status_code == 200:
            models = response.json()
            print("✓ Ollama is running!")
            print("Available models:")
            for model in models.get('models', []):
                print(f"  - {model['name']}")
            return models.get('models', [])
        else:
            print("✗ Ollama is not responding properly")
            return []
    except requests.exceptions.ConnectionError:
        print("✗ Ollama is not running. Please start Ollama first.")
        print("  You can start it by running: ollama serve")
        return []
    except Exception as e:
        print(f"✗ Error checking Ollama status: {e}")
        return []

# Check Ollama status
available_models = check_ollama_status()

✓ Ollama is running!
Available models:


In [8]:
# Function to create embeddings with available models
def create_ollama_embeddings(model_name=None):
    """Create Ollama embeddings with the specified model or find a suitable one."""
    if not available_models:
        print("No models available. Please install a model first.")
        return None
    
    # If no model specified, try to find a suitable embedding model
    if not model_name:
        # Look for common embedding models
        embedding_models = ['nomic-embed-text', 'all-minilm', 'mxbai-embed-large']
        for model in available_models:
            model_name = model['name']
            if any(embed_model in model_name.lower() for embed_model in embedding_models):
                print(f"Using available embedding model: {model_name}")
                break
        else:
            # If no embedding model found, use the first available model
            model_name = available_models[0]['name']
            print(f"No specific embedding model found. Using: {model_name}")
    
    try:
        embeddings = OllamaEmbeddings(model=model_name)
        print(f"✓ Successfully created embeddings with model: {model_name}")
        return embeddings
    except Exception as e:
        print(f"✗ Error creating embeddings with {model_name}: {e}")
        return None

# Create embeddings
embeddings = create_ollama_embeddings()

No models available. Please install a model first.


In [10]:
# Display embeddings configuration
if embeddings:
    print("Embeddings Configuration:")
    print(f"  Model: {embeddings.model}")
    print(f"  Base URL: {embeddings.base_url}")
    print(f"  Embed Instruction: {embeddings.embed_instruction}")
    print(f"  Query Instruction: {embeddings.query_instruction}")
else:
    print("No embeddings object created. Please check Ollama setup.")

No embeddings object created. Please check Ollama setup.


In [11]:
# Test embeddings with sample documents
if embeddings:
    print("Testing document embeddings...")
    try:
        sample_texts = ["this is a test", "this is a test 2", "machine learning is fascinating"]
        r1 = embeddings.embed_documents(sample_texts)
        print(f"✓ Successfully created {len(r1)} embeddings")
        print(f"  Each embedding has {len(r1[0])} dimensions")
        print(f"  First embedding (first 5 values): {r1[0][:5]}")
        
        # Test query embedding
        print("\nTesting query embedding...")
        query_embedding = embeddings.embed_query("this is a query")
        print(f"✓ Query embedding created successfully")
        print(f"  Query embedding dimensions: {len(query_embedding)}")
        print(f"  Query embedding (first 5 values): {query_embedding[:5]}")
        
    except Exception as e:
        print(f"✗ Error creating embeddings: {e}")
        print("This might be because:")
        print("  1. The model doesn't support embeddings")
        print("  2. Ollama is not running")
        print("  3. The model needs to be pulled first")
        print("\nTo fix this, try:")
        print("  1. Start Ollama: ollama serve")
        print("  2. Pull an embedding model: ollama pull nomic-embed-text")
else:
    print("Cannot test embeddings - no embeddings object available")

Cannot test embeddings - no embeddings object available


# Ollama Embeddings with LangChain

This notebook demonstrates how to use Ollama embeddings with LangChain.

## Prerequisites:
1. **Install Ollama**: Download from [ollama.ai](https://ollama.ai)
2. **Start Ollama**: Run `ollama serve` in terminal
3. **Pull an embedding model**: Run `ollama pull nomic-embed-text`

## Recommended Embedding Models:
- `nomic-embed-text` - Good general purpose embeddings
- `all-minilm` - Lightweight and fast
- `mxbai-embed-large` - High quality embeddings

## Common Issues:
- **Model not found**: Pull the model first with `ollama pull <model-name>`
- **Connection error**: Make sure Ollama is running with `ollama serve`
- **404 error**: The model might not support embeddings (use embedding-specific models)
